# Deep Learning for Movie Recommendation
In this notebook, I'll build a deep learning model movie recommendations system on the MovieLens 1M dataset.

This will be my 3rd attempt doing this.
* In the [1st attempt](https://github.com/khanhnamle1994/movielens/blob/master/Content_Based_and_Collaborative_Filtering_Models.ipynb), I tried out content-based and memory-based collaboratice filtering, which rely on the calculation of users and movies' similarity scores. As no training or optimization is involved, these are easy to use approaches. But their performance decrease when we have sparse data which hinders scalability of these approaches for most of the real-world problems.
* In the [2nd attempt](https://github.com/khanhnamle1994/movielens/blob/master/SVD_Model.ipynb), I tried out a matrix factorization model-based collaborative filtering approach called Singular Vector Decomposition, which reduces the dimension of the dataset and gives low-rank approximation of user tastes and preferences.

In this post, I will use a Deep Learning / Neural Network approach that is up and coming with recent development in machine learning and AI technologies.

![movixai](images/movixai.png)

## Loading Datasets
Similar to what I did for the previous notebooks, I loaded the 3 datasets into 3 dataframes: *ratings*, *users*, and *movies*. Additionally, to make it easy to use series from the *ratings* dataframe as training inputs and output to the Keras model, I set *max_userid* as the max value of user_id in the ratings and *max_movieid* as the max value of movie_id in the ratings.

In [ ]:
# Import libraries
%matplotlib inline
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reading ratings file
ratings = pd.read_csv('ratings.csv', sep='\t', encoding='latin-1',
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])
max_userid = ratings['user_id'].drop_duplicates().max()
max_movieid = ratings['movie_id'].drop_duplicates().max()

# Reading ratings file
users = pd.read_csv('users.csv', sep='\t', encoding='latin-1',
                    usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])

# Reading ratings file
movies = pd.read_csv('movies.csv', sep='\t', encoding='latin-1',
                     usecols=['movie_id', 'title', 'genres'])

## Matrix Factorization for Collaborative Filtering
I have discussed this extensively in my 2nd notebook, but want to revisit it here as the baseline approach. The idea behind matrix factorization models is that attitudes or preferences of a user can be determined by a small number of hidden factors. We can call these factors as **Embeddings**.

Intuitively, we can understand embeddings as low dimensional hidden factors for movies and users. For e.g. say we have 3 dimensional embeddings for both movies and users.

For instance, for movie A, the 3 numbers in the movie embedding matrix represent 3 different characteristics about the movie, such as:
* How recent is the movie A?
* How much special effects are in movie A?
* How CGI-driven is movie A?

For user B, the 3 numbers in the user embedding matrix represent:
* How much does user B like Drama movie?
* How likely does user B to give a 5-star rating?
* How often does user B watch movies?

![matrix-factorization](images/matrix-factorization.png)

I definitely would want to create a training and validation set and optimize the number of embeddings by minimizing the RMSE. Intuitively, the RMSE will decrease on the training set as the number of embeddings increases (because I'm approximating the original ratings matrix with a higher rank matrix). Here I create a training set by shuffling randomly the values from the original ratings dataset.

In [ ]:
# Create training set
shuffled_ratings = ratings.sample(frac=1., random_state=42)

# Shuffling users
Users = shuffled_ratings['user_emb_id'].values
print ('Users:', Users, ', shape =', Users.shape)

# Shuffling movies
Movies = shuffled_ratings['movie_emb_id'].values
print ('Movies:', Movies, ', shape =', Movies.shape)

# Shuffling ratings
Ratings = shuffled_ratings['rating'].values
print ('Ratings:', Ratings, ', shape =', Ratings.shape)

Users: [231 124 376 ...   9 122 376] , shape = (60213,)
Movies: [ 470 3752 3638 ... 2639 2874 1269] , shape = (60213,)
Ratings: [5 5 4 ... 5 2 4] , shape = (60213,)


## Deep Learning Model
The idea of using deep learning is similar to that of Model-Based Matrix Factorization. In matrix factorizaion, we decompose our original sparse matrix into product of 2 low rank orthogonal matrices. For deep learning implementation, we don’t need them to be orthogonal, we want our model to learn the values of embedding matrix itself. The user latent features and movie latent features are looked up from the embedding matrices for specific movie-user combination. These are the input values for further linear and non-linear layers. We can pass this input to multiple relu, linear or sigmoid layers and learn the corresponding weights by any optimization algorithm (Adam, SGD, etc.).

### Build the Model
I created a sparse matrix factoring algorithm in Keras for my model in CFModel. Here are the main components:

* A left embedding layer that creates a Users by Latent Factors matrix.
* A right embedding layer that creates a Movies by Latent Factors matrix.
* When the input to these layers are (i) a user id and (ii) a movie id, they'll return the latent factor vectors for the user and the movie, respectively.
* A merge layer that takes the dot product of these two latent vectors to return the predicted rating.


In [ ]:
class CFModel(Model):
    def __init__(self, max_userid, max_movieid, K_FACTORS):
        # Input variables
        user_input = Input(shape=(1,))
        movie_input = Input(shape=(1,))

        # Embedding layers
        P = Embedding(max_userid, K_FACTORS, input_length=1)(user_input)
        Q = Embedding(max_movieid, K_FACTORS, input_length=1)(movie_input)

        # Reshape the embeddings
        user_latent = Reshape((K_FACTORS,))(P)  # (K_FACTORS,)
        movie_latent = Reshape((K_FACTORS,))(Q)  # (K_FACTORS,)

        # Merge the embeddings using concatenation instead of Merge
        merged_vector = concatenate([user_latent, movie_latent], axis=-1)

        # Prediction layer
        prediction = Dense(1, activation='sigmoid')(merged_vector)

        # Define the model
        # Pass inputs and outputs to super().__init__()
        super(CFModel, self).__init__(inputs=[user_input, movie_input],
                                     outputs=prediction)

    def rate(self, user_id, item_id):
        return self.predict([np.array([user_id]), np.array([item_id])])[0][0]

In [ ]:
# Import Keras libraries
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

In [73]:
# Define constants
K_FACTORS = 100 # The number of dimensional embeddings for movies and users
# Pick a real user from the ratings dataset
TEST_USER = ratings['user_id'].sample(1).values[0]
print(f"Using TEST_USER = {TEST_USER}")

Using TEST_USER = 212


I then compile the model using Mean Squared Error (MSE) as the loss function and the AdaMax learning algorithm.

In [74]:
# Define model
model = CFModel(max_userid, max_movieid, K_FACTORS)
# Compile the model using MSE as the loss function and the AdaMax learning algorithm
model.compile(loss='mse', optimizer='adamax')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


### Train the Model
Now I need to train the model. This step will be the most-time consuming one. I spitted the training and validataion data with ratio of 90/10.

In [75]:
# Callbacks monitor the validation loss
# Save the model weights each time the validation loss has improved
callbacks = [EarlyStopping('val_loss', patience=2),
             ModelCheckpoint('weights.h5', save_best_only=True)]

# Use 30 epochs, 90% training data, 10% validation data
history = model.fit([Users, Movies], Ratings, epochs=30, validation_split=.1, verbose=2, callbacks=callbacks)

Epoch 1/30


1694/1694 - 28s - 17ms/step - loss: 8.7230 - val_loss: 8.1169
Epoch 2/30


1694/1694 - 4s - 3ms/step - loss: 8.0545 - val_loss: 8.0767
Epoch 3/30


1694/1694 - 5s - 3ms/step - loss: 8.0394 - val_loss: 8.0726
Epoch 4/30


1694/1694 - 4s - 2ms/step - loss: 8.0375 - val_loss: 8.0719
Epoch 5/30


1694/1694 - 5s - 3ms/step - loss: 8.0372 - val_loss: 8.0718
Epoch 6/30


1694/1694 - 5s - 3ms/step - loss: 8.0372 - val_loss: 8.0718
Epoch 7/30


1694/1694 - 5s - 3ms/step - loss: 8.0371 - val_loss: 8.0717
Epoch 8/30


1694/1694 - 5s - 3ms/step - loss: 8.0371 - val_loss: 8.0717
Epoch 9/30


1694/1694 - 4s - 3ms/step - loss: 8.0371 - val_loss: 8.0717
Epoch 10/30
1694/1694 - 5s - 3ms/step - loss: 8.0371 - val_loss: 8.0717
Epoch 11/30
1694/1694 - 10s - 6ms/step - loss: 8.0371 - val_loss: 8.0717


### Root Mean Square Error
During the training process above, I saved the model weights each time the validation loss has improved. Thus, I can use that value to calculate the best validation Root Mean Square Error.

In [77]:
# Show the best validation RMSE
min_val_loss, idx = min((val, idx) for (idx, val) in enumerate(history.history['val_loss']))
print ('Minimum RMSE at epoch', '{:d}'.format(idx+1), '=', '{:.4f}'.format(math.sqrt(min_val_loss)))

Minimum RMSE at epoch 9 = 2.8411


Te next step is to actually predict the ratings a random user will give to a random movie. Below I apply the freshly trained deep learning model for all the users and all the movies, using 100 dimensional embeddings for each of them. I also load pre-trained weights from *[weights.h5](https://github.com/khanhnamle1994/movielens/blob/master/weights.h5)* for the model.

In [78]:
# Use the pre-trained model
trained_model = CFModel(max_userid, max_movieid, K_FACTORS)
# Load weights
trained_model.load_weights('weights.h5')

As mentioned above, my random test user is has ID 212.

In [79]:
# Pick a random test user
users[users['user_id'] == TEST_USER]

,user_id,gender,zipcode,age_desc,occ_desc
211,212,M,53714,25-34,self-employed


Here I define the function to predict user's rating of unrated items, using the *rate* function inside the CFModel.

In [80]:
# Function to predict the ratings given User ID and Movie ID
def predict_rating(user_id, movie_id):
    return trained_model.rate(user_id - 1, movie_id - 1)

Here I show the top 20 movies that user 2000 has already rated, including the *predictions* column showing the values that used 2000 would have rated based on the newly defined *predict_rating* function.

In [81]:
user_ratings = ratings[ratings['user_id'] == TEST_USER][['user_id', 'movie_id', 'rating']]
user_ratings['prediction'] = user_ratings.apply(lambda x: float(predict_rating(TEST_USER, x['movie_id'])), axis=1)
user_ratings.sort_values(by='rating',
                         ascending=False).merge(movies,
                                                on='movie_id',
                                                how='inner',
                                                suffixes=['_u', '_m']).head(20)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━

,user_id,movie_id,rating,prediction,title,genres
0,212,1259,5,1.0,Stand by Me (1986),Adventure|Comedy|Drama
1,212,589,5,1.0,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
2,212,786,5,1.0,Eraser (1996),Action|Thriller
3,212,1608,5,1.0,Air Force One (1997),Action|Thriller
4,212,1370,5,1.0,Die Hard 2 (1990),Action|Thriller
5,212,3104,5,1.0,Midnight Run (1988),Action|Adventure|Comedy|Crime
6,212,474,5,1.0,In the Line of Fire (1993),Action|Thriller
7,212,1036,5,1.0,Die Hard (1988),Action|Thriller
8,212,2797,5,1.0,Big (1988),Comedy|Fantasy
9,212,440,5,1.0,Dave (1993),Comedy|Romance


No surpise that these top movies all have 5-start rating. Some of the prediction values seem off (those with value 3.7, 3.8, 3.9 etc.).

### Recommend Movies
Here I make a recommendation list of unrated 20 movies sorted by prediction value for user 2000. Let's see it.

In [82]:
recommendations = ratings[ratings['movie_id'].isin(user_ratings['movie_id']) == False][['movie_id']].drop_duplicates()
recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(TEST_USER, x['movie_id']), axis=1)
recommendations.sort_values(by='prediction',
                          ascending=False).merge(movies,
                                                 on='movie_id',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).head(20)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━

,movie_id,prediction,title,genres
0,2219,1.0,Murder! (1930),Mystery|Thriller
1,2762,1.0,"Sixth Sense, The (1999)",Thriller
2,1022,1.0,Cinderella (1950),Animation|Children's|Musical
3,1836,1.0,"Last Days of Disco, The (1998)",Drama
4,783,1.0,"Hunchback of Notre Dame, The (1996)",Animation|Children's|Musical
5,1907,1.0,Mulan (1998),Animation|Children's
6,588,1.0,Aladdin (1992),Animation|Children's|Comedy|Musical
7,1566,1.0,Hercules (1997),Adventure|Animation|Children's|Comedy|Musical
8,3186,1.0,"Girl, Interrupted (1999)",Drama
9,2294,1.0,Antz (1998),Animation|Children's


## Conclusion
In this notebook, I showed how to use a simple deep learning approach to build a recommendation engine for the MovieLens 1M dataset. This model performed better than all the approaches I attempted before (content-based, user-item similarity collaborative filtering, SVD). I can certainly improve this model's performance by making it deeper with more linear and non-linear layers.